In [32]:
import numpy as np
import pandas as pd
import warnings
import os
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [33]:
def _get_cents_ratio(f1, f2):
    return np.log2(f1 / f2) * 1200

def _get_cents_edo_step(edo):
    f2 = 1
    f1 = 2 ** (1/edo)
    return round(_get_cents_ratio(f1, f2), 10)

def _express_ratio(row):
    return f"{int(row['f1'])} : {int(row['f2'])}"

def _get_cents_row(row):
    return _get_cents_ratio(row['f1'], row['f2'])

In [34]:
NUM_HARMONICS = 256
harmonics = range(1, NUM_HARMONICS + 1)
just_ratios_adj = pd.DataFrame(np.zeros((NUM_HARMONICS, NUM_HARMONICS)), columns=harmonics)
just_ratios_adj.index = harmonics
just_ratios_edge = just_ratios_adj.stack().reset_index().rename({'level_0': 'f1', 'level_1': 'f2', 0: 'cents'}, axis=1)
just_ratios_edge = just_ratios_edge[just_ratios_edge['f1'] > just_ratios_edge['f2']]

just_ratios_edge['ratio'] = just_ratios_edge.apply(_express_ratio, axis=1)
just_ratios_edge['cents'] = just_ratios_edge.apply(_get_cents_row, axis=1)

In [35]:
def closest_approx_to_edo(cents: float, edo: int):
    # Returns (num steps, remainder):
    edo_step_size = _get_cents_edo_step(edo)
    num_steps_abs = cents / edo_step_size
    num_steps_rounded = np.round(num_steps_abs)
    offset_steps = round((num_steps_abs - num_steps_rounded), 5)

    return (num_steps_rounded, offset_steps)

def apply_aprox_to_edo(row, edo: int):
    return closest_approx_to_edo(row['cents'], edo)

In [36]:
ERROR_CENTS = 5

def process_EDO(EDO):

    concordance_shells_str = ""

    just_ratios_edge[f'{EDO}_edo_approx'] = just_ratios_edge.apply(lambda x: apply_aprox_to_edo(x, EDO), axis=1)
    just_ratios_edge[[f'{EDO}_edo_steps', f'{EDO}_edo_error_steps']] = just_ratios_edge[f'{EDO}_edo_approx'].to_list()
    just_ratios_edge[f'{EDO}_edo_error_cents'] = just_ratios_edge[f'{EDO}_edo_error_steps'] * _get_cents_edo_step(EDO)
    just_ratios_edge.drop([f'{EDO}_edo_approx'], axis=1, inplace=True)

    # VERY ROUGH APPROXIMATION OF ENTROPY = sqrt(f1 * f2) / (error + 1)
    just_ratios_edge['approx_dyadic_entropy'] = np.sqrt(just_ratios_edge['f1'] * just_ratios_edge['f2']) * (np.abs(just_ratios_edge[f'{EDO}_edo_error_cents']) + 1)
    just_ratios_edge['intra_octave_steps'] = (just_ratios_edge[f'{EDO}_edo_steps'] % EDO).astype(int)
    just_ratios_edge[f'intra_octave_cents_{EDO}edo'] = just_ratios_edge['intra_octave_steps'] * _get_cents_edo_step(EDO)
    df = just_ratios_edge[np.abs(just_ratios_edge[f'{EDO}_edo_error_cents']) < ERROR_CENTS]

    def find_primodal_concordance_shells(p):
        # List of harmonics to check
        f1s = []
        for i in range(256):
            cond1 = i > p
            cond2 = (i < 2 * p) | (i % 2 == 1)
            if cond1 * cond2:
                f1s.append(i)
        csdf = df[(df['f2'] == p) & (df['f1'].isin(f1s))].copy()
        csdf['relative_error_change_cents'] = csdf[f'{EDO}_edo_error_cents'].diff()
        return csdf.reset_index(drop=True)
    
    def format_concordance_df(csdf, p):
        shell = [p] + csdf['f1'].tolist()
        shellstr = f"{shell}".replace("[", "").replace("]", "").replace(", ", " : ")

        details = "None"

        if len(csdf) > 0:
            details = csdf[['ratio', f'{EDO}_edo_error_cents', f'intra_octave_cents_{EDO}edo', 'intra_octave_steps', 'relative_error_change_cents']].to_string(index=False)

        return f"{EDO}-edo concordance shell around /{p}: {shellstr}" + "\n\n" + "Details\n" + f"{details}" + "\n\n"

    primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
    
    for p in primes:
        csdf = find_primodal_concordance_shells(p)
        concordance_shells_str += format_concordance_df(csdf, p)

    concordance_shells_str += "----------------------------------\n"

    return concordance_shells_str

In [37]:
concordance_shells_all_edos = ""

for EDO in range(5, 54):
    concordance_shells_all_edos += process_EDO(EDO)

In [38]:
output_path = os.path.expanduser("~/Downloads/concordance_shells_all_edos_5c.txt")

with open(output_path, "w") as text_file:
    text_file.write(concordance_shells_all_edos)